In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

from scipy import stats

In [5]:
df = pd.read_csv(r'C:\Users\HP\Desktop\Oreilly\SalesForecast/train.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')
df.set_index('date', inplace=True, drop=True)
df.head()

,id,store_nbr,family,sales,onpromotion
date,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0.0
2013-01-01,1194,42,CELEBRATION,0.0,0.0
2013-01-01,1193,42,BREAD/BAKERY,0.0,0.0
2013-01-01,1192,42,BOOKS,0.0,0.0
2013-01-01,1191,42,BEVERAGES,0.0,0.0


In [6]:
df2 = df.copy()
num_lags = 3 # number of lags and window lenghts for mean aggregation
delay = 1 # predict target one step ahead
for column in df2:
    for lag in range(1,num_lags+1):
        df2[column + '_lag' + str(lag)] = df2[column].shift(lag*-1-(delay-1))
        if column != 'family':
            df2[column + '_avg_window_length' + str(lag+1)] = df2[column].shift(-1-(delay-1)).rolling(window=lag+1,center=False).mean().shift(1-(lag+1))

df2.dropna(inplace=True) 

mask = (df2.columns.str.contains('sales') | df2.columns.str.contains('lag') | df2.columns.str.contains('window'))
df_processed = df2[df2.columns[mask]]

# the columns in the processed dataframe
df_processed.columns

Index(['sales', 'id_lag1', 'id_avg_window_length2', 'id_lag2',
       'id_avg_window_length3', 'id_lag3', 'id_avg_window_length4',
       'store_nbr_lag1', 'store_nbr_avg_window_length2', 'store_nbr_lag2',
       'store_nbr_avg_window_length3', 'store_nbr_lag3',
       'store_nbr_avg_window_length4', 'family_lag1', 'family_lag2',
       'family_lag3', 'sales_lag1', 'sales_avg_window_length2', 'sales_lag2',
       'sales_avg_window_length3', 'sales_lag3', 'sales_avg_window_length4',
       'onpromotion_lag1', 'onpromotion_avg_window_length2',
       'onpromotion_lag2', 'onpromotion_avg_window_length3',
       'onpromotion_lag3', 'onpromotion_avg_window_length4'],
      dtype='object')

In [7]:
mask = df_processed.columns.str.contains('sales')
df_processed[df_processed.columns[mask]].head(10)

,sales,sales_lag1,sales_avg_window_length2,sales_lag2,sales_avg_window_length3,sales_lag3,sales_avg_window_length4
date,,,,,,,
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_processed.reset_index(drop=True,inplace=True)
df_train = df_processed.loc[:int(df_processed.shape[0]*0.8),:]
df_test = df_processed.loc[int(df_processed.shape[0]*0.8):,:]

In [9]:
h2o.init(nthreads=-1)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,12 mins 50 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,2 months and 8 days
H2O_cluster_name:,H2O_from_python_HP_e40ez8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.288 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [10]:
hf_train = h2o.H2OFrame(df_train)
hf_test = h2o.H2OFrame(df_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
hf_train.describe()

Rows:2400708
Cols:28




,sales,id_lag1,id_avg_window_length2,id_lag2,id_avg_window_length3,id_lag3,id_avg_window_length4,store_nbr_lag1,store_nbr_avg_window_length2,store_nbr_lag2,store_nbr_avg_window_length3,store_nbr_lag3,store_nbr_avg_window_length4,family_lag1,family_lag2,family_lag3,sales_lag1,sales_avg_window_length2,sales_lag2,sales_avg_window_length3,sales_lag3,sales_avg_window_length4,onpromotion_lag1,onpromotion_avg_window_length2,onpromotion_lag2,onpromotion_avg_window_length3,onpromotion_lag3,onpromotion_avg_window_length4
type,real,int,real,int,real,int,real,int,real,int,real,int,real,enum,enum,enum,real,real,real,real,real,real,int,real,int,real,int,real
mins,0.0,1.0,2.5,1.0,3.0,1.0,3.5,1.0,1.0,1.0,1.0,1.0,1.0,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,328.2725144943659,1200354.6407555605,1200355.1406545485,1200355.6405535364,1200355.6405535364,1200356.6403515139,1200356.1404525249,27.501743235745455,27.50174261092978,27.501741986114098,27.5017419861141,27.501740736482738,27.50174136129841,,,,328.2727835816518,328.27301393036555,328.27324427907956,328.2730912686602,328.27324594525476,328.27320616533746,28.783453048017503,28.783503449815633,28.783553851613767,28.783520666958793,28.783555101245145,28.78353812708583
maxs,124717.0,2401689.0,2401687.5,2401689.0,2401687.0,2401689.0,2401686.5,54.0,54.0,54.0,54.0,54.0,54.0,,,,124717.0,62462.5,124717.0,41732.85766666666,124717.0,31549.89325000003,944.0,663.0,944.0,452.0,944.0,362.75
sigma,1024.9142543573776,693025.093217693,693025.0927374611,693025.0945887365,693025.0927239178,693025.095956896,693025.0928098889,15.585499562792696,15.22930990065691,15.585498520603617,15.066761187682495,15.585497478414371,14.949411814465103,,,,1024.9142529730634,719.679176265996,1024.9143539875456,591.2803765107835,1024.914353457132,515.6733521310387,88.43727755706921,63.0565755224256,88.43738266831974,52.00684429741229,88.43738228279948,45.732154078147225
zeros,841721,0,0,0,0,0,0,0,0,0,0,0,0,,,,841720,405006,841719,198129,841718,27954,1265815,1004537,1265814,935747,1265813,911228
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.0,1194.0,1193.5,1193.0,1193.0,1192.0,1192.5,42.0,42.0,42.0,42.0,42.0,42.0,CELEBRATION,BREAD/BAKERY,BOOKS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1193.0,1192.5,1192.0,1192.0,1191.0,1191.5,42.0,42.0,42.0,42.0,42.0,42.0,BREAD/BAKERY,BOOKS,BEVERAGES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1192.0,1191.5,1191.0,1191.0,1190.0,1190.5,42.0,42.0,42.0,42.0,42.0,42.0,BOOKS,BEVERAGES,BEAUTY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
y = 'sales'
X = hf_train.columns
X.remove(y)

In [13]:
aml = H2OAutoML(max_runtime_secs = 600,
                seed = 42)
aml.train(x = X, 
          y = y,
          training_frame = hf_train,
          leaderboard_frame = hf_test)

AutoML progress: |█Failed polling AutoML progress log: [WinError 32] Het proces heeft geen toegang tot het bestand omdat het door een ander
proces wordt gebruikt: 'C:\\Users\\HP\\AppData\\Local\\Temp\\tmpimn6ju6m.csv'
█████
12:35:59.570: AutoML: XGBoost is not available; skipping it.
12:35:59.570: Step 'best_of_family_xgboost' not defined in provider 'StackedEnsemble': skipping it.
12:35:59.570: Step 'all_xgboost' not defined in provider 'StackedEnsemble': skipping it.

█████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_2_20211215_123559

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 132868.64886760275
RMSE: 364.51152089831504
MAE: 106.65203380188791
RMSLE: NaN
R^2: 0.8640894827310865
Mean Residual Deviance: 132868.64886760275
Null degrees of freedom: 9983
Residual degrees of freedom: 9982
N

In [14]:
lb = aml.leaderboard

lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_1_AutoML_2_20211215_123559,289456,538.011,289456,155.673,nan
StackedEnsemble_BestOfFamily_2_AutoML_2_20211215_123559,298482,546.335,298482,148.068,nan
StackedEnsemble_AllModels_1_AutoML_2_20211215_123559,298482,546.335,298482,148.068,nan
GBM_1_AutoML_2_20211215_123559,510338,714.38,510338,265.2,2.69189
DRF_1_AutoML_2_20211215_123559,584163,764.305,584163,161.791,1.24679
GBM_2_AutoML_2_20211215_123559,1.5148e+06,1230.77,1.5148e+06,513.055,3.28929
GBM_4_AutoML_2_20211215_123559,1.6176e+06,1271.85,1.6176e+06,536.479,3.33686
GBM_3_AutoML_2_20211215_123559,1.66163e+06,1289.04,1.66163e+06,542.226,3.34751
GLM_1_AutoML_2_20211215_123559,1.87433e+06,1369.06,1.87433e+06,579.019,3.42795


In [15]:
leader_model = aml.leader

In [16]:
hf_test_predict = leader_model.predict(hf_test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [17]:
hf_test_predict.head(5)


predict
233.997
586.998
1324.48
48.6634
721.105


In [19]:
df_results = pd.DataFrame()
df_results['ground_truth'] = df_test['sales'].reset_index(drop=True)
df_results['predictions'] = h2o.as_list(hf_test_predict,use_pandas=True)
df_results.head()

,ground_truth,predictions
0,193.492,233.997459
1,646.000,586.997861
2,1106.000,1324.476131
3,4.000,48.663363
4,732.000,721.105172
